# Car Price Prediction Challenge

## Problem Statement:
Car price prediction is a regression problem where the goal is to predict the price of a car based on various 
features like brand, year, mileage, engine size, fuel type, transmission, and more. 
This problem is widely used in the automotive and finance industries to help buyers, sellers, and 
businesses make data-driven decisions.

https://www.kaggle.com/datasets/deepcontractor/car-price-prediction-challenge/data

## Import libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder,OrdinalEncoder 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer


from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils import shuffle
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import pearsonr
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

C:\Users\shamr\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
data = pd.read_csv('data/car_price_prediction.csv')
data.sample(4)

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
1096,45776291,90,-,HYUNDAI,Sonata,2013,Sedan,No,Petrol,2.4,100800 km,4,Automatic,Front,04-May,Left wheel,Black,8
19167,45799423,18817,1995,FORD,Transit,2003,Microbus,No,Diesel,2.4 Turbo,2147483647 km,4,Manual,Front,02-Mar,Left wheel,White,2
426,45768685,627,586,LEXUS,CT 200h,2012,Hatchback,Yes,Hybrid,1.8,192683 km,4,Automatic,Front,04-May,Left wheel,White,0
14195,45730312,24123,1099,SSANGYONG,REXTON,2010,Jeep,Yes,Diesel,2.7,113724 km,4,Automatic,Front,04-May,Left wheel,Black,4


In [3]:
data.columns

Index(['ID', 'Price', 'Levy', 'Manufacturer', 'Model', 'Prod. year',
       'Category', 'Leather interior', 'Fuel type', 'Engine volume', 'Mileage',
       'Cylinders', 'Gear box type', 'Drive wheels', 'Doors', 'Wheel', 'Color',
       'Airbags'],
      dtype='object')

## Remove unwanted columns

In [4]:
data = data.drop(['ID','Doors'],axis=1)

In [5]:
data.head(2)

,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Wheel,Color,Airbags
0,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6,Automatic,4x4,Left wheel,Silver,12
1,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6,Tiptronic,4x4,Left wheel,Black,8


In [6]:
data.shape

(19237, 16)

## check null values

In [7]:
data.isnull().sum()

Price               0
Levy                0
Manufacturer        0
Model               0
Prod. year          0
Category            0
Leather interior    0
Fuel type           0
Engine volume       0
Mileage             0
Cylinders           0
Gear box type       0
Drive wheels        0
Wheel               0
Color               0
Airbags             0
dtype: int64

## Drop duplicates

In [8]:
data.duplicated().sum()

3512

In [9]:
data = data.drop_duplicates()

In [10]:
data.shape

(15725, 16)

## Descriptive analysis

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15725 entries, 0 to 19236
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Price             15725 non-null  int64 
 1   Levy              15725 non-null  object
 2   Manufacturer      15725 non-null  object
 3   Model             15725 non-null  object
 4   Prod. year        15725 non-null  int64 
 5   Category          15725 non-null  object
 6   Leather interior  15725 non-null  object
 7   Fuel type         15725 non-null  object
 8   Engine volume     15725 non-null  object
 9   Mileage           15725 non-null  object
 10  Cylinders         15725 non-null  int64 
 11  Gear box type     15725 non-null  object
 12  Drive wheels      15725 non-null  object
 13  Wheel             15725 non-null  object
 14  Color             15725 non-null  object
 15  Airbags           15725 non-null  int64 
dtypes: int64(4), object(12)
memory usage: 2.0+ MB


In [12]:
#organize num and categorical features
cat_features=[]
num_features=[]
for i in data.columns:
    if data[i].dtype=='int64':
        num_features.append(i)
    elif data[i].dtype=='object':
        cat_features.append(i)

print("Numerical features: ",num_features)
print()
print("Categorical features: ",cat_features)

Numerical features:  ['Price', 'Prod. year', 'Cylinders', 'Airbags']

Categorical features:  ['Levy', 'Manufacturer', 'Model', 'Category', 'Leather interior', 'Fuel type', 'Engine volume', 'Mileage', 'Gear box type', 'Drive wheels', 'Wheel', 'Color']


In [13]:
data['Levy'].value_counts()

Levy
-       5684
891      387
765      387
639      340
640      336
        ... 
2177       1
5666       1
3156       1
1719       1
1901       1
Name: count, Length: 559, dtype: int64

In [14]:
data['Levy'] = data['Levy'].replace('-',0)
data['Levy'] = data['Levy'].astype(int)
data['Levy'] = data['Levy'].replace(0,np.nan)
print(data['Levy'].value_counts())

Levy
891.0     387
765.0     387
639.0     340
640.0     336
642.0     245
         ... 
2177.0      1
5666.0      1
3156.0      1
1719.0      1
1901.0      1
Name: count, Length: 558, dtype: int64


In [15]:
data['Manufacturer'].value_counts()

Manufacturer
HYUNDAI          3050
TOYOTA           2863
MERCEDES-BENZ    1451
CHEVROLET         968
FORD              956
                 ... 
TESLA               1
PONTIAC             1
SATURN              1
ASTON MARTIN        1
GREATWALL           1
Name: count, Length: 65, dtype: int64

In [16]:
data['Manufacturer'].unique()

array(['LEXUS', 'CHEVROLET', 'HONDA', 'FORD', 'HYUNDAI', 'TOYOTA',
       'MERCEDES-BENZ', 'OPEL', 'PORSCHE', 'BMW', 'JEEP', 'VOLKSWAGEN',
       'AUDI', 'RENAULT', 'NISSAN', 'SUBARU', 'DAEWOO', 'KIA',
       'MITSUBISHI', 'SSANGYONG', 'MAZDA', 'GMC', 'FIAT', 'INFINITI',
       'ALFA ROMEO', 'SUZUKI', 'ACURA', 'LINCOLN', 'VAZ', 'GAZ',
       'CITROEN', 'LAND ROVER', 'MINI', 'DODGE', 'CHRYSLER', 'JAGUAR',
       'ISUZU', 'SKODA', 'DAIHATSU', 'BUICK', 'TESLA', 'CADILLAC',
       'PEUGEOT', 'BENTLEY', 'VOLVO', 'სხვა', 'HAVAL', 'HUMMER', 'SCION',
       'UAZ', 'MERCURY', 'ZAZ', 'ROVER', 'SEAT', 'LANCIA', 'MOSKVICH',
       'MASERATI', 'FERRARI', 'SAAB', 'LAMBORGHINI', 'ROLLS-ROYCE',
       'PONTIAC', 'SATURN', 'ASTON MARTIN', 'GREATWALL'], dtype=object)

In [17]:
data[data['Manufacturer']=='სხვა']

,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Wheel,Color,Airbags
2358,25089,NaN,სხვა,IVECO DAYLY,2007,Microbus,No,Diesel,2.3 Turbo,328000 km,4,Manual,Rear,Left wheel,White,1
4792,9408,NaN,სხვა,GONOW,2005,Jeep,Yes,Petrol,2.3,102000 km,4,Manual,Rear,Left wheel,Silver,2


In [18]:
data['Manufacturer'] = data['Manufacturer'].replace('სხვა',np.nan)
data['Manufacturer'].unique()

array(['LEXUS', 'CHEVROLET', 'HONDA', 'FORD', 'HYUNDAI', 'TOYOTA',
       'MERCEDES-BENZ', 'OPEL', 'PORSCHE', 'BMW', 'JEEP', 'VOLKSWAGEN',
       'AUDI', 'RENAULT', 'NISSAN', 'SUBARU', 'DAEWOO', 'KIA',
       'MITSUBISHI', 'SSANGYONG', 'MAZDA', 'GMC', 'FIAT', 'INFINITI',
       'ALFA ROMEO', 'SUZUKI', 'ACURA', 'LINCOLN', 'VAZ', 'GAZ',
       'CITROEN', 'LAND ROVER', 'MINI', 'DODGE', 'CHRYSLER', 'JAGUAR',
       'ISUZU', 'SKODA', 'DAIHATSU', 'BUICK', 'TESLA', 'CADILLAC',
       'PEUGEOT', 'BENTLEY', 'VOLVO', nan, 'HAVAL', 'HUMMER', 'SCION',
       'UAZ', 'MERCURY', 'ZAZ', 'ROVER', 'SEAT', 'LANCIA', 'MOSKVICH',
       'MASERATI', 'FERRARI', 'SAAB', 'LAMBORGHINI', 'ROLLS-ROYCE',
       'PONTIAC', 'SATURN', 'ASTON MARTIN', 'GREATWALL'], dtype=object)

#### Conclusion - Manufacturer has nan values, it will be taken up further

In [19]:
data['Model'].value_counts()

Model
Prius                     946
Sonata                    821
Elantra                   720
Camry                     581
Santa FE                  486
                         ... 
1500,1600 Schtufenheck      1
E 420                       1
C 250 luxury                1
Passat sport                1
Prius C aqua                1
Name: count, Length: 1590, dtype: int64

In [20]:
data['Category'].unique()

array(['Jeep', 'Hatchback', 'Sedan', 'Microbus', 'Goods wagon',
       'Universal', 'Coupe', 'Minivan', 'Cabriolet', 'Limousine',
       'Pickup'], dtype=object)

In [21]:
data['Leather interior'].unique()

array(['Yes', 'No'], dtype=object)

In [22]:
data['Fuel type'].unique()

array(['Hybrid', 'Petrol', 'Diesel', 'CNG', 'Plug-in Hybrid', 'LPG',
       'Hydrogen'], dtype=object)

In [23]:
data['Prod. year'].value_counts()

Prod. year
2012    1672
2014    1582
2013    1580
2015    1237
2016    1215
2011    1213
2010    1100
2017     754
2008     566
2009     479
2007     455
2018     421
2005     389
2003     355
2004     349
2006     310
2002     284
2000     276
2001     253
2019     231
1999     205
1998     204
1997     151
1996     109
1995     102
2020      43
1994      41
1992      30
1993      23
1990      18
1988      12
1991      10
1986       6
1989       6
1987       5
1984       5
1985       5
1953       4
1983       3
1939       3
1978       2
1980       2
1965       2
1977       2
1964       2
1943       1
1976       1
1957       1
1974       1
1968       1
1947       1
1982       1
1981       1
1973       1
Name: count, dtype: int64

In [24]:
data['Cylinders'].value_counts()

Cylinders
4     12279
6      2429
8       605
5       165
3       107
2        42
12       38
1        37
10       12
16        5
7         4
9         1
14        1
Name: count, dtype: int64

In [25]:
import datetime
now_date=datetime.datetime.now()
data['Age']=now_date.year-data['Prod. year']
data=data.drop(['Prod. year'],axis=1)

data.sample()

,Price,Levy,Manufacturer,Model,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Wheel,Color,Airbags,Age
4159,6115,NaN,NISSAN,Skyline,Sedan,No,CNG,2.5,165000 km,6,Tiptronic,Rear,Right-hand drive,Silver,2,24


In [26]:
data['Age'].value_counts()

Age
13    1672
11    1582
12    1580
10    1237
9     1215
14    1213
15    1100
8      754
17     566
16     479
18     455
7      421
20     389
22     355
21     349
19     310
23     284
25     276
24     253
6      231
26     205
27     204
28     151
29     109
30     102
5       43
31      41
33      30
32      23
35      18
37      12
34      10
39       6
36       6
38       5
41       5
40       5
72       4
42       3
86       3
47       2
45       2
60       2
48       2
61       2
82       1
49       1
68       1
51       1
57       1
78       1
43       1
44       1
52       1
Name: count, dtype: int64

In [27]:
data['Engine volume'].unique()

array(['3.5', '3', '1.3', '2.5', '2', '1.8', '2.4', '4', '1.6', '3.3',
       '2.0 Turbo', '2.2 Turbo', '4.7', '1.5', '4.4', '3.0 Turbo',
       '1.4 Turbo', '3.6', '2.3', '1.5 Turbo', '1.6 Turbo', '2.2',
       '2.3 Turbo', '1.4', '5.5', '2.8 Turbo', '3.2', '3.8', '4.6', '1.2',
       '5', '1.7', '2.9', '0.5', '1.8 Turbo', '2.4 Turbo', '3.5 Turbo',
       '1.9', '2.7', '4.8', '5.3', '0.4', '2.8', '3.2 Turbo', '1.1',
       '2.1', '0.7', '5.4', '1.3 Turbo', '3.7', '1', '2.5 Turbo', '2.6',
       '1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.8', '0.2 Turbo', '5.7',
       '4.8 Turbo', '4.6 Turbo', '6.7', '6.2', '1.2 Turbo', '3.4',
       '1.7 Turbo', '6.3 Turbo', '2.7 Turbo', '4.3', '4.2', '2.9 Turbo',
       '0', '4.0 Turbo', '20', '3.6 Turbo', '0.3', '3.7 Turbo', '5.9',
       '5.5 Turbo', '0.2', '2.1 Turbo', '5.6', '6', '0.7 Turbo',
       '0.6 Turbo', '6.8', '4.5', '0.6', '7.3', '0.1', '1.0 Turbo', '6.3',
       '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo', '6.4',
       '3

### Create new feature - Turbo

In [28]:
data['Has_Turbo'] = data['Engine volume'].str.contains('Turbo',case=False)
data['Has_Turbo']

0        False
1        False
2        False
3        False
4        False
         ...  
19230    False
19232     True
19233    False
19234    False
19236    False
Name: Has_Turbo, Length: 15725, dtype: bool

In [29]:
data.sample(5)

,Price,Levy,Manufacturer,Model,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Wheel,Color,Airbags,Age,Has_Turbo
10450,5425,1095.0,SSANGYONG,Kyron,Jeep,Yes,Diesel,2,98329 km,4,Automatic,Front,Left wheel,Black,4,17,False
6924,17249,NaN,FIAT,500,Coupe,No,Petrol,1.6,43000 km,4,Automatic,Front,Left wheel,Black,10,9,False
1406,3136,259.0,CHEVROLET,Volt,Hatchback,Yes,Hybrid,1.4,112123 km,4,Automatic,Front,Left wheel,Silver,12,11,False
6215,51517,891.0,HYUNDAI,Santa FE,Jeep,Yes,Diesel,2,123898 km,4,Automatic,Front,Left wheel,White,4,9,False
18235,24775,456.0,TOYOTA,Camry se,Sedan,Yes,Hybrid,2.5,141000 km,6,Variator,Front,Left wheel,Black,10,10,False


#### Engine volume feature is modified or engineered

In [30]:

data['Engine volume'] = data['Engine volume'].str.replace("Turbo","")
data['Engine volume'].unique()

array(['3.5', '3', '1.3', '2.5', '2', '1.8', '2.4', '4', '1.6', '3.3',
       '2.0 ', '2.2 ', '4.7', '1.5', '4.4', '3.0 ', '1.4 ', '3.6', '2.3',
       '1.5 ', '1.6 ', '2.2', '2.3 ', '1.4', '5.5', '2.8 ', '3.2', '3.8',
       '4.6', '1.2', '5', '1.7', '2.9', '0.5', '1.8 ', '2.4 ', '3.5 ',
       '1.9', '2.7', '4.8', '5.3', '0.4', '2.8', '3.2 ', '1.1', '2.1',
       '0.7', '5.4', '1.3 ', '3.7', '1', '2.5 ', '2.6', '1.9 ', '4.4 ',
       '4.7 ', '0.8', '0.2 ', '5.7', '4.8 ', '4.6 ', '6.7', '6.2', '1.2 ',
       '3.4', '1.7 ', '6.3 ', '2.7 ', '4.3', '4.2', '2.9 ', '0', '4.0 ',
       '20', '3.6 ', '0.3', '3.7 ', '5.9', '5.5 ', '0.2', '2.1 ', '5.6',
       '6', '0.7 ', '0.6 ', '6.8', '4.5', '0.6', '7.3', '0.1', '1.0 ',
       '6.3', '4.5 ', '0.8 ', '4.2 ', '3.1', '5.0 ', '6.4', '3.9', '5.7 ',
       '0.9', '0.4 ', '5.4 ', '0.3 ', '5.2', '5.8', '1.1 '], dtype=object)

In [31]:
data['Engine volume'] = data['Engine volume'].str.strip()
data['Engine volume'] = data['Engine volume'].astype('float')
data['Engine volume'].unique()

array([ 3.5,  3. ,  1.3,  2.5,  2. ,  1.8,  2.4,  4. ,  1.6,  3.3,  2.2,
        4.7,  1.5,  4.4,  1.4,  3.6,  2.3,  5.5,  2.8,  3.2,  3.8,  4.6,
        1.2,  5. ,  1.7,  2.9,  0.5,  1.9,  2.7,  4.8,  5.3,  0.4,  1.1,
        2.1,  0.7,  5.4,  3.7,  1. ,  2.6,  0.8,  0.2,  5.7,  6.7,  6.2,
        3.4,  6.3,  4.3,  4.2,  0. , 20. ,  0.3,  5.9,  5.6,  6. ,  0.6,
        6.8,  4.5,  7.3,  0.1,  3.1,  6.4,  3.9,  0.9,  5.2,  5.8])

In [32]:
data['Mileage'].unique()

array(['186005 km', '192000 km', '200000 km', ..., '140607 km',
       '307325 km', '186923 km'], dtype=object)

In [33]:
data['Mileage'] = data['Mileage'].str.replace("km","")
data['Mileage'] = data['Mileage'].str.strip()
data['Mileage'] = data['Mileage'].astype('int')

data['Mileage'].unique()

array([186005, 192000, 200000, ..., 140607, 307325, 186923])

In [34]:
data['Gear box type'].unique()

array(['Automatic', 'Tiptronic', 'Variator', 'Manual'], dtype=object)

In [35]:
data['Drive wheels'].unique()

array(['4x4', 'Front', 'Rear'], dtype=object)

In [36]:
data['Wheel'].unique()

array(['Left wheel', 'Right-hand drive'], dtype=object)

In [37]:
data['Color'].unique()

array(['Silver', 'Black', 'White', 'Grey', 'Blue', 'Green', 'Red',
       'Sky blue', 'Orange', 'Yellow', 'Brown', 'Golden', 'Beige',
       'Carnelian red', 'Purple', 'Pink'], dtype=object)

In [38]:
data['Color'].value_counts()

Color
Black            3835
White            3611
Silver           3149
Grey             1919
Blue             1247
Red               567
Green             288
Orange            204
Carnelian red     177
Brown             172
Golden            142
Beige             134
Sky blue          122
Yellow             94
Purple             39
Pink               25
Name: count, dtype: int64

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15725 entries, 0 to 19236
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Price             15725 non-null  int64  
 1   Levy              10041 non-null  float64
 2   Manufacturer      15723 non-null  object 
 3   Model             15725 non-null  object 
 4   Category          15725 non-null  object 
 5   Leather interior  15725 non-null  object 
 6   Fuel type         15725 non-null  object 
 7   Engine volume     15725 non-null  float64
 8   Mileage           15725 non-null  int32  
 9   Cylinders         15725 non-null  int64  
 10  Gear box type     15725 non-null  object 
 11  Drive wheels      15725 non-null  object 
 12  Wheel             15725 non-null  object 
 13  Color             15725 non-null  object 
 14  Airbags           15725 non-null  int64  
 15  Age               15725 non-null  int64  
 16  Has_Turbo         15725 non-null  bool   
dty

### Check null values

In [40]:
data.isnull().sum()

Price                  0
Levy                5684
Manufacturer           2
Model                  0
Category               0
Leather interior       0
Fuel type              0
Engine volume          0
Mileage                0
Cylinders              0
Gear box type          0
Drive wheels           0
Wheel                  0
Color                  0
Airbags                0
Age                    0
Has_Turbo              0
dtype: int64

In [41]:
data.isnull().sum()/data.shape[0]*100

Price                0.000000
Levy                36.146264
Manufacturer         0.012719
Model                0.000000
Category             0.000000
Leather interior     0.000000
Fuel type            0.000000
Engine volume        0.000000
Mileage              0.000000
Cylinders            0.000000
Gear box type        0.000000
Drive wheels         0.000000
Wheel                0.000000
Color                0.000000
Airbags              0.000000
Age                  0.000000
Has_Turbo            0.000000
dtype: float64

### conclusion -
1. Impluting column 'Levy' as there are 36% null data
2. Imputing manufacturer column using forward fill

In [42]:
imputer = SimpleImputer(strategy='median')
data['Levy'] = imputer.fit_transform(data[['Levy']])

In [43]:
data.isnull().sum()/data.shape[0]*100

Price               0.000000
Levy                0.000000
Manufacturer        0.012719
Model               0.000000
Category            0.000000
Leather interior    0.000000
Fuel type           0.000000
Engine volume       0.000000
Mileage             0.000000
Cylinders           0.000000
Gear box type       0.000000
Drive wheels        0.000000
Wheel               0.000000
Color               0.000000
Airbags             0.000000
Age                 0.000000
Has_Turbo           0.000000
dtype: float64

In [44]:
data.head(2)

,Price,Levy,Manufacturer,Model,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Wheel,Color,Airbags,Age,Has_Turbo
0,13328,1399.0,LEXUS,RX 450,Jeep,Yes,Hybrid,3.5,186005,6,Automatic,4x4,Left wheel,Silver,12,15,False
1,16621,1018.0,CHEVROLET,Equinox,Jeep,No,Petrol,3.0,192000,6,Tiptronic,4x4,Left wheel,Black,8,14,False


In [45]:
data.Manufacturer.unique()

array(['LEXUS', 'CHEVROLET', 'HONDA', 'FORD', 'HYUNDAI', 'TOYOTA',
       'MERCEDES-BENZ', 'OPEL', 'PORSCHE', 'BMW', 'JEEP', 'VOLKSWAGEN',
       'AUDI', 'RENAULT', 'NISSAN', 'SUBARU', 'DAEWOO', 'KIA',
       'MITSUBISHI', 'SSANGYONG', 'MAZDA', 'GMC', 'FIAT', 'INFINITI',
       'ALFA ROMEO', 'SUZUKI', 'ACURA', 'LINCOLN', 'VAZ', 'GAZ',
       'CITROEN', 'LAND ROVER', 'MINI', 'DODGE', 'CHRYSLER', 'JAGUAR',
       'ISUZU', 'SKODA', 'DAIHATSU', 'BUICK', 'TESLA', 'CADILLAC',
       'PEUGEOT', 'BENTLEY', 'VOLVO', nan, 'HAVAL', 'HUMMER', 'SCION',
       'UAZ', 'MERCURY', 'ZAZ', 'ROVER', 'SEAT', 'LANCIA', 'MOSKVICH',
       'MASERATI', 'FERRARI', 'SAAB', 'LAMBORGHINI', 'ROLLS-ROYCE',
       'PONTIAC', 'SATURN', 'ASTON MARTIN', 'GREATWALL'], dtype=object)

In [46]:
data['Manufacturer'].fillna(method='ffill', inplace=True)  

C:\Users\shamr\AppData\Local\Temp\ipykernel_26300\3720621773.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Manufacturer'].fillna(method='ffill', inplace=True)
C:\Users\shamr\AppData\Local\Temp\ipykernel_26300\3720621773.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['Manufacturer'].fillna(method='ffill', inplace=True)


In [47]:
data.isnull().sum()

Price               0
Levy                0
Manufacturer        0
Model               0
Category            0
Leather interior    0
Fuel type           0
Engine volume       0
Mileage             0
Cylinders           0
Gear box type       0
Drive wheels        0
Wheel               0
Color               0
Airbags             0
Age                 0
Has_Turbo           0
dtype: int64

In [48]:
#organize num and categorical features
cat_features=[]
num_features=[]
for i in data.columns:
    if data[i].dtype=='int64' or data[i].dtype=='float64' :
        num_features.append(i)
    elif data[i].dtype=='object':
        cat_features.append(i)

print("Numerical features: ",num_features)
print()
print("Categorical features: ",cat_features)

Numerical features:  ['Price', 'Levy', 'Engine volume', 'Cylinders', 'Airbags', 'Age']

Categorical features:  ['Manufacturer', 'Model', 'Category', 'Leather interior', 'Fuel type', 'Gear box type', 'Drive wheels', 'Wheel', 'Color']


### Outlier detection

In [49]:
numeric_columns = data.select_dtypes(exclude=['object']).columns

# Calculate the IQR for each numeric column
Q1 = data[num_features].quantile(0.25)
Q3 = data[num_features].quantile(0.75)
IQR = Q3 - Q1

# Define the bounds to filter out outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(lower_bound)
print("-------------------------")
print(upper_bound)


Price           -16228.5
Levy               572.0
Engine volume        0.5
Cylinders            4.0
Airbags             -8.0
Age                  2.0
dtype: float64
-------------------------
Price            47119.5
Levy              1036.0
Engine volume        3.7
Cylinders            4.0
Airbags             24.0
Age                 26.0
dtype: float64


In [50]:
# Filter out the outliers
data = data[~((data[num_features] < lower_bound) | (data[num_features] > upper_bound)).any(axis=1)]

# Display the cleaned data
data.shape

(8732, 17)

In [51]:
for feat in cat_features:
    print("-----------",feat,"------------")
    print(data[feat].value_counts())

----------- Manufacturer ------------
Manufacturer
HYUNDAI          2283
TOYOTA           1768
CHEVROLET         599
HONDA             512
FORD              511
MERCEDES-BENZ     372
SSANGYONG         363
NISSAN            349
VOLKSWAGEN        338
KIA               285
OPEL              196
SUBARU            170
MITSUBISHI        170
BMW               148
LEXUS              99
MAZDA              92
AUDI               77
DAEWOO             59
JEEP               52
SUZUKI             40
DODGE              37
FIAT               34
MINI               28
RENAULT            27
SKODA              18
VAZ                14
PEUGEOT            11
CHRYSLER            9
BUICK               8
JAGUAR              7
SCION               7
ACURA               5
LAND ROVER          5
DAIHATSU            5
GMC                 5
CITROEN             5
INFINITI            4
VOLVO               3
ALFA ROMEO          3
SEAT                2
CADILLAC            2
LINCOLN             2
UAZ                 2
HAV

In [52]:
print(list(data['Manufacturer'].value_counts()))

[2283, 1768, 599, 512, 511, 372, 363, 349, 338, 285, 196, 170, 170, 148, 99, 92, 77, 59, 52, 40, 37, 34, 28, 27, 18, 14, 11, 9, 8, 7, 7, 5, 5, 5, 5, 5, 4, 3, 3, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1]


In [53]:
df_counts = data['Manufacturer'].value_counts().reset_index()
df_counts.columns = ['Manufacturer', 'Count']

# Filter where Count < 50
df_counts_filtered = df_counts[df_counts['Count'] < 50]
print(df_counts_filtered['Manufacturer'])


19        SUZUKI
20         DODGE
21          FIAT
22          MINI
23       RENAULT
24         SKODA
25           VAZ
26       PEUGEOT
27      CHRYSLER
28         BUICK
29        JAGUAR
30         SCION
31         ACURA
32    LAND ROVER
33      DAIHATSU
34           GMC
35       CITROEN
36      INFINITI
37         VOLVO
38    ALFA ROMEO
39          SEAT
40      CADILLAC
41       LINCOLN
42           UAZ
43         HAVAL
44        LANCIA
45       MERCURY
46          SAAB
47        SATURN
48           GAZ
Name: Manufacturer, dtype: object


In [54]:
def manufacturer(company):
    if company in list(df_counts_filtered['Manufacturer']):
        return "OTHER"
    else:
        return company
    
data['Manufacturer'] = list(map(manufacturer,data['Manufacturer']))

In [55]:
data['Manufacturer'].value_counts()

Manufacturer
HYUNDAI          2283
TOYOTA           1768
CHEVROLET         599
HONDA             512
FORD              511
MERCEDES-BENZ     372
SSANGYONG         363
NISSAN            349
VOLKSWAGEN        338
OTHER             289
KIA               285
OPEL              196
SUBARU            170
MITSUBISHI        170
BMW               148
LEXUS              99
MAZDA              92
AUDI               77
DAEWOO             59
JEEP               52
Name: count, dtype: int64

### Split X and Y

In [56]:
data.columns

Index(['Price', 'Levy', 'Manufacturer', 'Model', 'Category',
       'Leather interior', 'Fuel type', 'Engine volume', 'Mileage',
       'Cylinders', 'Gear box type', 'Drive wheels', 'Wheel', 'Color',
       'Airbags', 'Age', 'Has_Turbo'],
      dtype='object')

In [57]:
y = data[['Price']]
y.head(4)

,Price
2,8467
3,3607
5,39493
6,1803


In [58]:
col = [i for i in range(1,data.shape[1])]
X = data.iloc[:,col]
X

,Levy,Manufacturer,Model,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Wheel,Color,Airbags,Age,Has_Turbo
2,779.0,HONDA,FIT,Hatchback,No,Petrol,1.3,200000,4,Variator,Front,Right-hand drive,Black,2,19,False
3,862.0,FORD,Escape,Jeep,Yes,Hybrid,2.5,168966,4,Automatic,4x4,Left wheel,White,0,14,False
5,891.0,HYUNDAI,Santa FE,Jeep,Yes,Diesel,2.0,160931,4,Automatic,Front,Left wheel,White,4,9,False
6,761.0,TOYOTA,Prius,Hatchback,Yes,Hybrid,1.8,258909,4,Automatic,Front,Left wheel,White,12,15,False
7,751.0,HYUNDAI,Sonata,Sedan,Yes,Petrol,2.4,216118,4,Automatic,Front,Left wheel,Grey,12,12,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19230,645.0,TOYOTA,Prius,Hatchback,Yes,Hybrid,1.8,307325,4,Automatic,Front,Left wheel,Silver,12,14,False
19232,779.0,MERCEDES-BENZ,CLK 200,Coupe,Yes,CNG,2.0,300000,4,Manual,Rear,Left wheel,Silver,5,26,True
19233,831.0,HYUNDAI,Sonata,Sedan,Yes,Petrol,2.4,161600,4,Tiptronic,Front,Left wheel,Red,8,14,False
19234,836.0,HYUNDAI,Tucson,Jeep,Yes,Diesel,2.0,116365,4,Automatic,Front,Left wheel,Grey,4,15,False


### Final cat and num features

In [59]:
#organize num and categorical features
cat_features=[]
num_features=[]
for i in X.columns:
    if X[i].dtype=='int64' or  X[i].dtype=='float':
        num_features.append(i)
    elif X[i].dtype=='object':
        cat_features.append(i)

print("Numerical features: ",num_features)
print()
print("Categorical features: ",cat_features)

Numerical features:  ['Levy', 'Engine volume', 'Cylinders', 'Airbags', 'Age']

Categorical features:  ['Manufacturer', 'Model', 'Category', 'Leather interior', 'Fuel type', 'Gear box type', 'Drive wheels', 'Wheel', 'Color']


## Feature selection

## 1. Correlation Analysis for numerical features

In [60]:
corr = X[num_features].corr()*100
corr

,Levy,Engine volume,Cylinders,Airbags,Age
Levy,100.000000,19.501152,NaN,-0.647319,4.477549
Engine volume,19.501152,100.000000,NaN,12.798156,-8.080803
Cylinders,NaN,NaN,NaN,NaN,NaN
Airbags,-0.647319,12.798156,NaN,100.000000,-24.202044
Age,4.477549,-8.080803,NaN,-24.202044,100.000000


#### Conclusion - 
Cylinders can be removed, as it has only 1 value , Removing Cylinders

In [61]:
# correlation analysis

correlations = {}
for col in num_features:
    corr, _ = pearsonr(X[col], y)
    correlations[col] = corr

C:\Users\shamr\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [62]:
correlations

{'Levy': array([0.02636497404558653], dtype=object),
 'Engine volume': array([0.1254882190885494], dtype=object),
 'Cylinders': nan,
 'Airbags': array([-0.1152523366238422], dtype=object),
 'Age': array([-0.41772907286072697], dtype=object)}

## 2.chisquare test for categorical 

In [63]:
# Chi-Square Test
chi2_result = {}

for col in cat_features:
    contingency_table = pd.crosstab(X[col], y['Price'])
    chi2,p,_,_ = chi2_contingency(contingency_table)
    chi2_result[col]= (chi2, p)    

print("\nChi-Square Test:")
for key, value in chi2_result.items():
    print(f"{key}: chi2={value[0]:.4f}, p-value={value[1]:.4f}")


Chi-Square Test:
Manufacturer: chi2=37721.4711, p-value=0.0000
Model: chi2=824925.3334, p-value=1.0000
Category: chi2=18159.3435, p-value=0.0000
Leather interior: chi2=3226.7613, p-value=0.0000
Fuel type: chi2=10216.3101, p-value=0.0000
Gear box type: chi2=4905.2134, p-value=0.0000
Drive wheels: chi2=2890.3500, p-value=0.9301
Wheel: chi2=2568.5590, p-value=0.0000
Color: chi2=19148.1960, p-value=1.0000


#### Conclusion - 
Color, Drive wheels, Model has no relationship with price label (Accepting Null Hypothesis) , so it can be removed

## 3. Anova test - to test categorical variables

In [64]:
# ANOVA Test
anova_results = {}
for col in cat_features:
    categories = X[col].unique()
    groups = [y[X[col]==category] for category in categories]
    f_val, p_val = f_oneway(*groups)
    anova_results[col] = (f_val, p_val)

print("\nANOVA Test:")
for key, value in anova_results.items():
    print(f"{key}: F = {value[0]}, p-value = {value[1]}")


ANOVA Test:
Manufacturer: F = [92.7691889], p-value = [0.]
Model: F = [7.94680355], p-value = [0.]
Category: F = [189.48595315], p-value = [0.]
Leather interior: F = [326.87610526], p-value = [9.28411762e-72]
Fuel type: F = [355.2696598], p-value = [0.]
Gear box type: F = [69.7040332], p-value = [1.53602358e-44]
Drive wheels: F = [15.05301629], p-value = [2.97718584e-07]
Wheel: F = [555.37765113], p-value = [4.21007925e-119]
Color: F = [13.24063142], p-value = [1.07795114e-33]


In [65]:
# drop model, color , drive wheels , cylinders as it has high p values

In [66]:
X = X.drop(['Model','Color','Drive wheels','Cylinders'],axis=1)
X.head(2)

,Levy,Manufacturer,Category,Leather interior,Fuel type,Engine volume,Mileage,Gear box type,Wheel,Airbags,Age,Has_Turbo
2,779.0,HONDA,Hatchback,No,Petrol,1.3,200000,Variator,Right-hand drive,2,19,False
3,862.0,FORD,Jeep,Yes,Hybrid,2.5,168966,Automatic,Left wheel,0,14,False


In [67]:
#organize num and categorical features
cat_features=[]
num_features=[]
for i in X.columns:
    if X[i].dtype=='int64' or  X[i].dtype=='float':
        num_features.append(i)
    elif X[i].dtype=='object':
        cat_features.append(i)

print("Numerical features: ",num_features)
print()
print("Categorical features: ",cat_features)

Numerical features:  ['Levy', 'Engine volume', 'Airbags', 'Age']

Categorical features:  ['Manufacturer', 'Category', 'Leather interior', 'Fuel type', 'Gear box type', 'Wheel']


## Column Transformer to transform for classic Regression models

In [68]:
preprocessor = ColumnTransformer([
    ('num',StandardScaler(),num_features),
    ('cat',OneHotEncoder(),cat_features)
])

#transformed_data = preprocessor.fit_transform(X)
transformed_data = preprocessor.fit_transform(X).toarray()

In [69]:
transformed_data

array([[ 0.18565201, -1.77533295, -1.11493802, ...,  1.        ,
         0.        ,  1.        ],
       [ 1.0600503 ,  1.61072227, -1.64331656, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.36556296,  0.19986593, -0.58655948, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 0.73346781,  1.328551  ,  0.4701976 , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.7861424 ,  0.19986593, -0.58655948, ...,  0.        ,
         1.        ,  0.        ],
       [-0.08825589,  1.328551  ,  1.52695468, ...,  0.        ,
         1.        ,  0.        ]])

In [70]:
# Get new column names
cat_columns = preprocessor.named_transformers_['cat'].get_feature_names_out(cat_features)
num_columns = num_features
all_columns = np.concatenate([num_columns,cat_columns])


# Convert to DataFrame
transformed_df = pd.DataFrame(transformed_data, columns=all_columns)

In [71]:
transformed_df.head()

,Levy,Engine volume,Airbags,Age,Manufacturer_AUDI,Manufacturer_BMW,Manufacturer_CHEVROLET,Manufacturer_DAEWOO,Manufacturer_FORD,Manufacturer_HONDA,...,Fuel type_Hybrid,Fuel type_LPG,Fuel type_Petrol,Fuel type_Plug-in Hybrid,Gear box type_Automatic,Gear box type_Manual,Gear box type_Tiptronic,Gear box type_Variator,Wheel_Left wheel,Wheel_Right-hand drive
0,0.185652,-1.775333,-1.114938,1.157139,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,1.060050,1.610722,-1.643317,0.038979,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,1.365563,0.199866,-0.586559,-1.079180,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,-0.003977,-0.364477,1.526955,0.262611,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,-0.109326,1.328551,1.526955,-0.408285,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [72]:
#y data 
scaler = StandardScaler()
y = scaler.fit_transform(y)
y

array([[-0.70589024],
       [-1.1519292 ],
       [ 2.14160042],
       ...,
       [-0.04380691],
       [ 0.91315774],
       [-1.43983541]])

## Train test split 

In [73]:
X_train, X_test, y_train, y_test = train_test_split(transformed_df,y,test_size=0.25,random_state=42)
print("X train : ",X_train.shape)
print("X test : ",X_test.shape)
print("y train : ",y_train.shape)
print("y test : ",y_test.shape)

X train :  (6549, 48)
X test :  (2183, 48)
y train :  (6549, 1)
y test :  (2183, 1)


In [74]:
X_train.head()

,Levy,Engine volume,Airbags,Age,Manufacturer_AUDI,Manufacturer_BMW,Manufacturer_CHEVROLET,Manufacturer_DAEWOO,Manufacturer_FORD,Manufacturer_HONDA,...,Fuel type_Hybrid,Fuel type_LPG,Fuel type_Petrol,Fuel type_Plug-in Hybrid,Gear box type_Automatic,Gear box type_Manual,Gear box type_Tiptronic,Gear box type_Variator,Wheel_Left wheel,Wheel_Right-hand drive
6985,-1.879192,-0.928819,-0.586559,0.038979,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3847,0.185652,-1.210990,0.998576,-0.184653,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
198,0.206722,-1.493162,0.470198,-1.526444,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
7828,0.185652,-0.928819,0.470198,-0.184653,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1097,0.185652,-0.364477,1.526955,0.486243,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


## Model Building

## Linear Regression

In [75]:
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions

y_pred = model.predict(X_test)

In [76]:
# Model evaluation
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred):.2f}")
print(f"R² Score: {r2_score(y_test, y_pred):.2f}")

Mean Absolute Error: 0.55
Mean Squared Error: 0.51
R² Score: 0.49


## KNN  Regression

In [77]:
knn = KNeighborsRegressor(n_neighbors=3)

# Train the model
knn.fit(X_train, y_train)

# Make predictions
y_pred = knn.predict(X_test)
y_train_pred = knn.predict(X_train)


In [78]:
# Model evaluation
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred):.2f}")
print(f"Trainings R² Score: {r2_score(y_train, y_train_pred):.2f}")
print(f"R² Score: {r2_score(y_test, y_pred):.2f}")

Mean Absolute Error: 0.40
Mean Squared Error: 0.34
Trainings R² Score: 0.80
R² Score: 0.65


## SVR

In [79]:
svr = SVR(kernel='rbf', C=100, epsilon=1.0, gamma=0.1)

# Train the model
svr.fit(X_train, y_train)

# Make predictions
y_pred = svr.predict(X_test)


C:\Users\shamr\anaconda3\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [80]:
# Model evaluation
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred):.2f}")
print(f"R² Score: {r2_score(y_test, y_pred):.2f}")

Mean Absolute Error: 0.51
Mean Squared Error: 0.42
R² Score: 0.57


## ML Pipeline

In [81]:
# Define Models
from sklearn.pipeline import Pipeline
lr_model = Pipeline(steps=[ ("model", LinearRegression())])
ada_model = Pipeline(steps=[ ("Regressor", AdaBoostRegressor())])
gra_model = Pipeline(steps=[("Regressor", GradientBoostingRegressor())])
rf_model = Pipeline(steps=[("Regressor", RandomForestRegressor())])
dt_model = Pipeline(steps=[ ("Regressor", DecisionTreeRegressor())])
xgb_model = Pipeline(steps=[ ("Regressor", XGBRegressor())])



estimators = [('lr', lr_model),('rf', rf_model),('gd',gra_model),('ada', ada_model),('dt', dt_model),('xgb', xgb_model)]

stack_model = StackingRegressor(estimators= estimators, final_estimator= Ridge())
vote_model = VotingRegressor(estimators= estimators)


# HyperParam Tuning
param_grid = {'Regressor__n_estimators':[100,200],
              'Regressor__learning_rate':[0.01,0.1],
              'Regressor__max_depth':[3,4,5],
              'Regressor__subsample':[0.8,0.9,1.0],
             }


from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator=gra_model, param_grid=param_grid, cv=5,
                          scoring='r2', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
best_gb_model = grid_search.best_estimator_

# Train and Evaluate models
models = {
    "Linear Regression": lr_model,
    "Adaboost Regressor":  ada_model,
    "Gradient Boosting" : gra_model,
    "Random Forest ": rf_model,
    "Decision Tree": dt_model,
    "XGB Tree": xgb_model,
    "stack_model": stack_model,
    "vote_model" : vote_model}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = (mse, r2)
    print(f"{name}:")
    print(f"Mean Squared Erorr : {mse}")
    print(f"R^2 SCORE : {r2}")
    print()

# Comparing the models
best_model_name = max(results, key=lambda k: results[k][1])
best_mse, best_r2 = results[best_model_name]
print(f"The Best Model is {best_model_name} with r^2 score of {best_r2} and MSE of {best_mse}")

Fitting 5 folds for each of 36 candidates, totalling 180 fits


C:\Users\shamr\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


Linear Regression:
Mean Squared Erorr : 0.5057411778463817
R^2 SCORE : 0.4863091613584547



C:\Users\shamr\anaconda3\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Adaboost Regressor:
Mean Squared Erorr : 0.5586569612619291
R^2 SCORE : 0.4325616036138813



...

Gradient Boosting:
Mean Squared Erorr : 0.3264076455725697
R^2 SCORE : 0.668461607363686



C:\Users\shamr\anaconda3\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest :
Mean Squared Erorr : 0.2718601354324068
R^2 SCORE : 0.7238665406717262

Decision Tree:
Mean Squared Erorr : 0.377135287671894
R^2 SCORE : 0.6169365859618786

XGB Tree:
Mean Squared Erorr : 0.2651855795863622
R^2 SCORE : 0.7306460127422325



C:\Users\shamr\anaconda3\lib\site-packages\sklearn\ensemble\_stacking.py:970: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\shamr\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.80722e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\shamr\anaconda3\lib\site-packages\sklearn\ensemble\_voting.py:694: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


stack_model:
Mean Squared Erorr : 0.254602382713374
R^2 SCORE : 0.7413955651127637

vote_model:
Mean Squared Erorr : 0.2847864603801554
R^2 SCORE : 0.7107370289890151

The Best Model is stack_model with r^2 score of 0.7413955651127637 and MSE of 0.254602382713374
